In [1]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import requests
import json
import pandas as pd
from datetime import date,time,datetime,timedelta
from openpyxl import load_workbook
import itertools
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plotly.tools.set_credentials_file(username='rishabh.gupta.min15', api_key='sqmOt6Mv5NcdNM74FWm8')
auth_token_hiya="fc3ed86b62485a2b9a6a680477ea7e60"
auth_token_rawble="e8253e7e6017303a54f57ce28b9d209a"
organization_id_hiya="654806043"
organization_id_rawble = "667580392"
base_url = "https://books.zoho.com/api/v3"
end_points = {'invoices':'/invoices','crm':'/crm','contacts':'/contacts','account':'/account','bills':'/bills'}

In [3]:
page_number = 1
parameters={'authtoken':auth_token_hiya,'organization_id':organization_id_hiya}
parameters['page'] = page_number
df_list_inv = []
for i in itertools.count():
    parameters['page'] = page_number + i
    response = requests.get(base_url + end_points['invoices'],params = parameters)
    df_temp = pd.DataFrame(response.json()['invoices'])
    df_list_inv.append(df_temp)
    print(parameters)
    if(response.json()['page_context']['has_more_page'] != True):
        break
print(len(df_list_inv))

df_invoice_sheet = pd.concat(df_list_inv,axis=0,sort=False)
print(len(df_invoice_sheet))

{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 1}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 2}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 3}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 4}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 5}
5
908


In [4]:
df_new_inv = pd.DataFrame()
for i in range(len(df_invoice_sheet)):
    if(df_invoice_sheet.iloc[i]['status']=='paid' and df_invoice_sheet.iloc[i]['last_payment_date'] != ''):
        customer_name = df_invoice_sheet.iloc[i]['customer_name']
        difference_date  = (datetime.strptime(df_invoice_sheet.iloc[i]['last_payment_date'],'%Y-%m-%d').date()-datetime.strptime(df_invoice_sheet.iloc[i]['due_date'],'%Y-%m-%d').date()).days
        if (difference_date <0):
            difference_date = 0
        amount = df_invoice_sheet.iloc[i]['total']
        df_new_inv = df_new_inv.append({
            'customer_name':customer_name,'difference_days_avg':difference_date,'amount':amount,
        },ignore_index=True)
g = df_new_inv.groupby('customer_name')
df_new_inv["weighted_avg"] = (df_new_inv.difference_days_avg / g.amount.transform("sum")) * df_new_inv.amount
df_new_2_inv = pd.DataFrame(df_new_inv.groupby('customer_name').agg({'weighted_avg':'sum','difference_days_avg':['mean','max','min']}))
print(df_new_2_inv['difference_days_avg'])

                                                          mean    max    min
customer_name                                                               
 Zeal Herbs India                                    37.000000   37.0   37.0
AARSHVED NATURALS                                     2.333333    7.0    0.0
ABHISHEIK PHARMACEUTICALS                            38.700000   85.0   10.0
AETHON INTERNATIONAL LLP                             22.250000   39.0    0.0
AKUMS DRUGS & PHARMACEAUTICALS LTD (PLANT 2-LIQ...   15.000000   15.0   15.0
AKUMS DRUGS & PHARMACEUTICALS LTD (PLANT-1-SOLI...   46.500000   83.0   10.0
AKUMS DRUGS & PHARMACEUTICALS LTD( PLANT 5 -  C...   10.800000   16.0    8.0
ALTIS LIFE SCIENCES                                  21.739130   81.0    0.0
AMOGH PHARMACEUTICALS PRIVATE LIMITED                40.500000   65.0    0.0
AMSTER LABS                                          80.333333  115.0   12.0
ARELA HERBALS                                        16.666667   47.0    0.0

In [5]:
today = date.today()
week_start = today #+ timedelta(days=7-today.weekday())
first_day =week_start
week = timedelta(days=7)
week_end = week_start + week

In [6]:
a=(df_new_2_inv.weighted_avg['sum'])
print(a)

customer_name
 Zeal Herbs India                                            37.000000
AARSHVED NATURALS                                             3.898932
ABHISHEIK PHARMACEUTICALS                                    33.524511
AETHON INTERNATIONAL LLP                                     18.812500
AKUMS DRUGS & PHARMACEAUTICALS LTD (PLANT 2-LIQUID)          15.000000
AKUMS DRUGS & PHARMACEUTICALS LTD (PLANT-1-SOLID ORAL)       15.782246
AKUMS DRUGS & PHARMACEUTICALS LTD( PLANT 5 -  COSMETIC)      10.682748
ALTIS LIFE SCIENCES                                          15.215505
AMOGH PHARMACEUTICALS PRIVATE LIMITED                        37.983378
AMSTER LABS                                                  73.474636
ARELA HERBALS                                                10.885148
ATIYA HEALTHCARE PRIVATE LIMITED                              0.373839
AYURPRAKASH HERBAL PVT. LTD                                  52.335336
AYUU HEALTH CARE                                              0

In [7]:
#book = load_workbook('Projections.xlsx')
#writer = pd.ExcelWriter("Projections.xlsx", engine='openpyxl')
#writer.book = book
#writer.sheets = dict((ws.title, ws) for ws in book.worksheets)


df_wk1_inv = df_wk2_inv = df_wk3_inv = df_wk4_inv = pd.DataFrame()
four_wk_report_inv = [df_wk1_inv,df_wk2_inv,df_wk3_inv,df_wk4_inv]

df_invoices_2 = pd.DataFrame(columns=['customer_name','balance','age'])
for i in range(len(df_invoice_sheet)):
    
    customer_name = df_invoice_sheet.iloc[i]['customer_name'] 
    balance = df_invoice_sheet.iloc[i]['balance']
    
    age=" "
    
    if(df_invoice_sheet.iloc[i]['due_days'].split(" ")[0]=='Overdue'):
        
        age = int(df_invoice_sheet.iloc[i]['due_days'].split(" ")[2]) * 1
    elif(df_invoice_sheet.iloc[i]['due_days'].split(" ")[0]=='Due'):
        
        if(df_invoice_sheet.iloc[i]['due_days'].split(" ")[1]=='Today'):
            age=0
        else:
            age = int(df_invoice_sheet.iloc[i]['due_days'].split(" ")[2]) * -1
    
    df_invoices_2 = df_invoices_2.append({
        'customer_name':customer_name,
        'balance':balance,
        'age':age,
    },ignore_index=True)

for i in range(len(four_wk_report_inv)):
    df_temp = pd.DataFrame()
    for k in range(len(df_invoices_2)):
       
        customer_name = df_invoices_2.iloc[k]['customer_name']
        balance = df_invoices_2.iloc[k]['balance']            
        if(df_invoices_2.iloc[k]['age'] != ' '):
            new_age = int(df_invoices_2.iloc[k]['age']) + (i*7)
            weighted_avg=difference_days_avg = probability = "No record"
            if(new_age >= 0):
                minimum_ent = 0
                maximum_ent =0 
                
                if(not df_new_2_inv.weighted_avg[df_new_2_inv.index == customer_name]['sum'].empty):
                    weighted_avg = int(df_new_2_inv.weighted_avg[df_new_2_inv.index == customer_name]['sum'])
                
                if(not df_new_2_inv.difference_days_avg[df_new_2_inv.index == customer_name]['mean'].empty):
                    difference_days_avg = int(df_new_2_inv.difference_days_avg[df_new_2_inv.index == customer_name]['mean'])
                if(not df_new_2_inv.difference_days_avg[df_new_2_inv.index == customer_name]['min'].empty):
                    minimum_ent = int(df_new_2_inv.difference_days_avg[df_new_2_inv.index == customer_name]['min'])
                if(not df_new_2_inv.difference_days_avg[df_new_2_inv.index == customer_name]['max'].empty):
                    maximum_ent = int(df_new_2_inv.difference_days_avg[df_new_2_inv.index == customer_name]['max'])
                
                if(minimum_ent == maximum_ent) :
                    if (new_age<minimum_ent):
                        probability = "25% - 50%"
                    if(new_age>=minimum_ent):
                        probability = "max 100%"
                else:
                    if(new_age < minimum_ent):
                        probability = "0% - 25%"
                    if(new_age>=minimum_ent and new_age< difference_days_avg):
                        probability = "25% - 50%"
                    if(new_age >= difference_days_avg and new_age < (difference_days_avg + ((maximum_ent - difference_days_avg)/2))):
                        probability = "50% - 75%"
                    if(new_age >= (difference_days_avg + ((maximum_ent - difference_days_avg)/2)) and new_age < maximum_ent):
                        probability = "75% - 100%"
                    if(new_age >= maximum_ent):
                        probability = "max 100%"

                #if(weighted_avg_list):
                   # weighted_avg = math.ceil(weighted_avg_list)
                #if(difference_days_avg_list):
                    #difference_days_avg = math.ceil(difference_days_avg_list)

                        
                        
                df_temp = df_temp.append({
                    'Customer_Name': customer_name,
                    'Balance': balance,
                    'Age' : new_age,
                    'A_Weighted average':weighted_avg,
                    'A_Difference days average':difference_days_avg,
                    'probability': probability,
                    
                },ignore_index=True)
    four_wk_report_inv[i] = df_temp   
sum_recievable_wk = [0,0,0,0]
for j in range(len(four_wk_report_inv)):
    for i in range(len(four_wk_report_inv[j])):
        df_temp = four_wk_report_inv[j]
        sum_recievable_wk[j] = sum_recievable_wk[j] + df_temp.iloc[i]['Balance']
    

result1 = pd.concat(four_wk_report_inv, keys=['as of '+str(first_day),'as of '+str(first_day+week), 'as of '+str(first_day+(week+week)),'as of '+str(first_day+(week+week+week))],axis=1)
print(result1)

#result1.to_excel(writer, sheet_name='Sheet1')
#writer.save()

             as of 2018-07-04                                      \
    A_Difference days average A_Weighted average    Age   Balance   
0                          33                 56    2.0   43375.0   
1                   No record          No record  368.0   94510.0   
2                   No record          No record  272.0    2667.0   
3                   No record          No record   16.0    5310.0   
4                          33                 56   30.0   13700.0   
5                          20                 21    3.0     142.0   
6                   No record          No record    7.0   45700.0   
7                          35                 36    7.0    4720.0   
8                          21                 15    9.0   59331.0   
9                          38                 33   10.0  437740.0   
10                         38                 33   11.0    6920.0   
11                  No record          No record   41.0  332000.0   
12                  No record     

In [50]:
print(four_wk_report_inv[0])

    A_Difference days average A_Weighted average    Age    Balance  \
0                   No record          No record  367.0   94510.00   
1                   No record          No record  271.0    2667.00   
2                   No record          No record   15.0    5310.00   
3                          33                 56   29.0   13700.00   
4                          20                 21    2.0     142.00   
5                   No record          No record    6.0   45700.00   
6                          35                 36    6.0    4720.00   
7                          21                 15    8.0   59331.00   
8                          38                 33    9.0  437740.00   
9                          38                 33   10.0    6920.00   
10                  No record          No record   40.0  332000.00   
11                  No record          No record   40.0    3068.00   
12                          0                  0   40.0   48026.00   
13                  

In [14]:
df_graph_pivoted = four_wk_report_inv[0].pivot_table(index = "Customer_Name",columns = "probability",values="Balance",aggfunc = "sum")
data = []
for index,Customer_Name in df_graph_pivoted.iterrows():
    trace = go.Bar( x = df_graph_pivoted.columns, y = Customer_Name , name = index  )
    data.append(trace)
layout = go.Layout(
        title = "As of " +str(today),
        showlegend=True,
        barmode="stack",
xaxis = dict(
      
      title = "Probability"
    ),
    yaxis = dict(
      
      title = "Balance"
    ))
figure = go.Figure(data=data, layout=layout)
plotly.offline.plot(figure)
print(df_graph_pivoted)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [7]:
page_number = 1
df_list_bi = []
for i in itertools.count():
    parameters['page'] = page_number + i
    response = requests.get(base_url + end_points['bills'],params = parameters)
    df_temp = pd.DataFrame(response.json()['bills'])
    df_list_bi.append(df_temp)
    print(parameters)
    if(response.json()['page_context']['has_more_page'] != True):
        break
print(len(df_list_bi))

df_bill_sheet = pd.concat(df_list_bi,axis=0,sort=False)
print(len(df_bill_sheet))

{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 1}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 2}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 3}
{'authtoken': 'fc3ed86b62485a2b9a6a680477ea7e60', 'organization_id': '654806043', 'page': 4}
4
693


In [8]:
df_new_bi = pd.DataFrame()
for i in range(len(df_bill_sheet)):
    
    if(df_bill_sheet.iloc[i]['status']=='paid'):
        vendor_name = df_bill_sheet.iloc[i]['vendor_name']
        difference_date  = (datetime.strptime(str(df_bill_sheet.iloc[i]['last_modified_time']).split("T")[0],'%Y-%m-%d').date()-datetime.strptime(df_bill_sheet.iloc[i]['due_date'],'%Y-%m-%d').date()).days
        
        amount = df_bill_sheet.iloc[i]['total']
        df_new_bi = df_new_bi.append({
            'vendor_name':vendor_name,'difference_days_avg':difference_date,'amount':amount,
        },ignore_index=True)
    
g = df_new_bi.groupby('vendor_name')
df_new_bi["weighted_avg"] = (df_new_bi.difference_days_avg / g.amount.transform("sum")) * df_new_bi.amount
df_new_2_bi = pd.DataFrame(df_new_bi.groupby('vendor_name').agg({'weighted_avg':'sum','difference_days_avg':'mean'}))


In [9]:
today = date.today()
week_start = today #+ timedelta(days=7-today.weekday())
first_day =week_start
week = timedelta(days=7)
week_end = week_start + week

In [10]:
df_wk1_bi = df_wk2_bi = df_wk3_bi = df_wk4_bi = pd.DataFrame()
four_wk_report_bi = [df_wk1_bi,df_wk2_bi,df_wk3_bi,df_wk4_bi]

df_bill_2 = pd.DataFrame(columns=['vendor_name','balance','age'])
for i in range(len(df_bill_sheet)):
    
    vendor_name = df_bill_sheet.iloc[i]['vendor_name'] 
    balance = df_bill_sheet.iloc[i]['balance']
    
    age=" "
    
    if(df_bill_sheet.iloc[i]['due_days'].split(" ")[0]=='Overdue'):
        
        age = int(df_bill_sheet.iloc[i]['due_days'].split(" ")[2]) * 1
    elif(df_bill_sheet.iloc[i]['due_days'].split(" ")[0]=='Due'):
        
        if(df_bill_sheet.iloc[i]['due_days'].split(" ")[1]=='Today'):
            age=0
        else:
            age = int(df_bill_sheet.iloc[i]['due_days'].split(" ")[2]) * -1
    
    df_bill_2 = df_bill_2.append({
        'vendor_name':vendor_name,
        'balance':balance,
        'age':age,
    },ignore_index=True)

for i in range(len(four_wk_report_bi)):
    df_temp = pd.DataFrame()
    for k in range(len(df_bill_2)):
        vendor_name = df_bill_2.iloc[k]['vendor_name']
        balance = df_bill_2.iloc[k]['balance']
        if(df_bill_2.iloc[k]['age'] != ' '):
            new_age = int(df_bill_2.iloc[k]['age']) + (i*7)
            weighted_avg=difference_days_avg = "No record"
            if(new_age >= 0):
                for j in range(len(df_new_2_bi)):
                    weighted_avg_list = list(df_new_2_bi.weighted_avg[df_new_2_bi.index == vendor_name])
                    difference_days_avg_list = list(df_new_2_bi.difference_days_avg[df_new_2_bi.index == vendor_name])

                    if(weighted_avg_list):
                        weighted_avg = math.ceil(weighted_avg_list[0])
                    if(difference_days_avg_list):
                        difference_days_avg = math.ceil(difference_days_avg_list[0])

                        
                        
                df_temp = df_temp.append({
                    'Vendor Name': vendor_name,
                    'Balance': balance,
                    'Age' : new_age,
                    'A_Weighted average':weighted_avg,
                    'A_Difference days average':difference_days_avg,
                },ignore_index=True)
    four_wk_report_bi[i] = df_temp   

sum_payable_wk = [0,0,0,0]
for j in range(len(four_wk_report_bi)):
    for i in range(len(four_wk_report_bi[j])):
        df_temp = four_wk_report_bi[j]
        sum_payable_wk[j] = sum_payable_wk[j] + df_temp.iloc[i]['Balance']

result2 = pd.concat(four_wk_report_bi, keys=['as of '+str(first_day),'as of '+str(first_day+week), 'as of '+str(first_day+(week+week)),'as of '+str(first_day+(week+week+week))],axis=1)
print(sum_payable_wk)

[17281688.05, 17870719.050000004, 17929091.050000004, 18569663.940000005]


In [11]:
difference = [0,0,0,0]
for j in range(len(sum_payable_wk)):
    difference[j] = sum_recievable_wk[j] - sum_payable_wk[j]
print(difference)

[-6420508.840000002, -6581969.440000005, -6130678.840000007, -6485027.730000008]


In [21]:
df_graph = pd.DataFrame({'payables':sum_payable_wk,'recievables':sum_recievable_wk,'difference':difference,'week':['as of '+str(first_day),'as of '+str(first_day+week), 'as of '+str(first_day+(week+week)),'as of '+str(first_day+(week+week+week))]})
print(df_graph)

      payables  recievables  difference              week
0  17281688.05  10861179.21 -6420508.84  as of 2018-07-02
1  17870719.05  11288749.61 -6581969.44  as of 2018-07-09
2  17929091.05  11798412.21 -6130678.84  as of 2018-07-16
3  18569663.94  12084636.21 -6485027.73  as of 2018-07-23


In [22]:
data = df_graph.melt(id_vars =['week'],value_vars=['payables','recievables','difference'] )
print(data)

                week     variable        value
0   as of 2018-07-02     payables  17281688.05
1   as of 2018-07-09     payables  17870719.05
2   as of 2018-07-16     payables  17929091.05
3   as of 2018-07-23     payables  18569663.94
4   as of 2018-07-02  recievables  10861179.21
5   as of 2018-07-09  recievables  11288749.61
6   as of 2018-07-16  recievables  11798412.21
7   as of 2018-07-23  recievables  12084636.21
8   as of 2018-07-02   difference  -6420508.84
9   as of 2018-07-09   difference  -6581969.44
10  as of 2018-07-16   difference  -6130678.84
11  as of 2018-07-23   difference  -6485027.73


In [23]:
df_graph = pd.DataFrame({'payables':sum_payable_wk,'recievables':sum_recievable_wk,'difference':difference,'week':['as of '+str(first_day),'as of '+str(first_day+week), 'as of '+str(first_day+(week+week)),'as of '+str(first_day+(week+week+week))]}).set_index('week',drop=True)

In [14]:
import cufflinks as cf
import numpy as np

In [15]:
cf.set_config_file(offline=False, world_readable=True, theme='pearl')

In [16]:
df_graph.iplot(kind='histogram' )

In [32]:
data = [{
    'x': df_graph.index,
    'y': df_graph[col],
    'name': col,
    'type':'bar',
    'hoverinfo':
    
}  for col in df_graph.columns]

py.iplot(data)



In [18]:
data = [[{
    'x': df_graph.index,
    'y': df_graph[col],
    'name': col
    'type':'histogram'
}  for col in df_graph.columns]
  go.Histogram(
    y = y,
    x = x,
    name = "count"
  ),
  go.Histogram(
    histfunc = "sum",
    y = y,
    x = x,
    name = "sum"
  )
]

SyntaxError: invalid syntax (<ipython-input-18-e0dd87f50fa0>, line 5)

In [ ]:
trace1 = go.Histogram(
    y= df_graph['payables'],x=df_graph.index,
    opacity=0.75,histnorm='percent'
)
trace2 = go.Histogram(
    y = df_graph['recievables'],x=df_graph.index,
    opacity=0.75,histnorm='percent'
)
trace3 = go.Histogram(
y = df_graph['difference'],x= df_graph.index,opacity=1,histnorm='percent')

data = [trace1,trace2,trace3]
py.iplot(data)
print(data)

In [29]:
result1.iplot(kind='scatter',x='balance')

KeyError: 'balance'